# 基于PyTorch的文本分类项目模型与训练代码讲解

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_32_基于PyTorch的文本分类项目模型与训练代码讲解：

https://www.bilibili.com/video/BV1eD4y1F7o4/?spm_id_from=pageDriver&vd_source=18e91d849da09d846f771c89a366ed40

***数据集***

IMDB Dataset of 50K Movie Reviews:

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext.datasets import IMDB
# conda install -c pytorch torchtext==${PyTorch所对应的版本} torchdata
from torchtext.datasets.imdb import NUM_LINES
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

import sys
import os
import logging
logging.basicConfig(
    level=logging.WARN,
    stream=sys.stdout,
    format="%(asctime)s (%(module)s:%(lineno)d) %(levelname)s: %(message)s",
)

# step1: 编写GCNN模型代码

In [2]:
VOCAB_SIZE = 15000

class GCNN(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, embedding_dim=64, num_class=2):
        super(GCNN, self).__init__()
        
        self.embedding_table = nn.Embedding(vocab_size, embedding_dim)
        nn.init.xavier_uniform_(self.embedding_table.weight)
        
        self.conv_A_1 = nn.Conv1d(embedding_dim, 64, 15, stride=7)
        self.conv_B_1 = nn.Conv1d(embedding_dim, 64, 15, stride=7)
        
        self.conv_A_2 = nn.Conv1d(64, 64, 15, stride=7)
        self.conv_B_2 = nn.Conv1d(64, 64, 15, stride=7)
        
        self.output_linear1 = nn.Linear(64, 128)
        self.output_linear2 = nn.Linear(128, num_class)
        
    def forward(self, word_index):
        # 定义GCN网络的算子操作流程，基于句子单词ID输入得到分类logitsshuchu
        
        # 1. 通过word_index得到word_embedding
        # word_index shape:[bs, max_seq_len, vocab_size]
        word_embedding = self.embedding_table(word_index)  # [bs, max_seq_len, embedding_dim]
        
        # 2. 编写第一层 1D 门卷积模块
        word_embedding = word_embedding.transpose(1,2)  # [bs, embedding_dim, max_seq_len]
        A = self.conv_A_1(word_embedding)
        B = self.conv_B_1(word_embedding)
        H = A * torch.sigmoid(B)  # [bs, 64, max_seq_len]
        
        A = self.conv_A_2(H)
        B = self.conv_B_2(H)
        H = A * torch.sigmoid(B)  # [bs, 64, max_seq_len]
        
        # 3. 池化并经过全连接层
        pool_output = torch.mean(H, dim=-1)  # 平均池化，得到[bs, 64]
        linear1_output = self.output_linear1(pool_output)
        logits = self.output_linear2(linear1_output) # [bs, 2]
        
        return logits
    
class TextClassificationModel(nn.Module):
    """ 简单版 embedding+DNN 模型 """
    
    def __init__(self, vocab_size=VOCAB_SIZE, embed_dim=64, num_class=2):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        
    def forward(self, token_index):
        embedded = self.embedding(token_index)  # shape: [bs, embedding_dim]
        return self.fc(embedded)
    

# step2 构建 IMDB DataLoader

In [3]:
# step2 构建 IMDB DataLoader

BATCH_SIZE = 64 

def yield_tokens(train_data_iter, tokenizer):
    for i, sample in enumerate(train_data_iter):
        label, comment = sample
        yield tokenizer(comment)
        
train_data_iter = IMDB(root='.data', split='train')  # Dataset类型的对象
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(yield_tokens(train_data_iter, tokenizer), min_freq=20, specials=["<unk>"])
vocab.set_default_index(0)
print(f"单词表大小：{len(vocab)}")

# 对 dataloader 生成的 minibatch 做后处理。将文本转化为张量
def collate_fn(batch):
    target = []
    token_index = []
    max_length = 0
    for i, (label, comment) in enumerate(batch):
        tokens = tokenizer(comment)
        
        token_index.append(vocab(tokens))
        if len(tokens) > max_length:
            max_length = len(tokens)
        
        if label == "pos":
            target.append(0)
        else:
            target.append(1)
    
    token_index = [index + [0]*(max_length-len(index)) for index in token_index]
    return (torch.tensor(target).to(torch.int64), torch.tensor(token_index).to(torch.int32))
            

D:\anaconda3\envs\swin\lib\site-packages\torch\utils\data\datapipes\utils\common.py:25: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  "Lambda function is not supported for pickle, please use "
D:\anaconda3\envs\swin\lib\site-packages\torch\utils\data\datapipes\iter\selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


单词表大小：13351


# step3 编写训练代码

In [4]:
def train(train_data_loader, eval_data_loader, model, optimizer, num_epoch, log_step_interval, save_step_interval, eval_step_interval,
          save_path, resume=""):
    """ 此处 data_loader 是 map-style dataset """
    start_epoch = 0
    start_step = 0
    if resume != "":
        # 加载之前训过的模型的参数文件
        logging.warning(f"loading from {resume}")
        checkpoint = torch.load(resume)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        start_step = checkpoint['step']
        
    for epoch_index in range(start_epoch, num_epoch):
        ema_loss = 0.  # 指数移动平均 loss
        num_batches = len(train_data_loader)
        
        for batch_index, (target, token_index) in enumerate(train_data_loader):
            optimizer.zero_grad()
            step = num_batches*(epoch_index) + batch_index + 1
            logits = model(token_index)
            bce_loss = F.binary_cross_entropy(torch.sigmoid(logits), F.one_hot(target, num_classes=2).to(torch.float32))
            ema_loss = 0.9*ema_loss + 0.1*bce_loss
            bce_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.1)  # 梯度截断
            optimizer.step()  # 梯度更新
            
            if step % log_step_interval == 0:
                logging.warning(f"epoch_index: {epoch_index}, batch_index: {batch_index}, ema_loss: {ema_loss}")
                
            if step % save_step_interval == 0:
                os.makedirs(save_path, exist_ok=True)
                save_file = os.path.join(save_path, f"step_{step}.pt")
                torch.save({
                    'epoch': epoch_index,
                    'step': step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': bce_loss,
                }, save_file)
                logging.warning(f"checkpoint has been saved in {save_file}")
                
            if step % eval_step_interval == 0:
                logging.warning("start to do evaluation...")
                model.eval()
                ema_eval_loss = 0
                total_acc_account = 0
                total_account = 0
                for eval_batch_index, (eval_target, eval_token_index) in enumerate(eval_data_loader):
                    total_account += eval_target.shape[0]
                    eval_logits = model(eval_token_index)
                    total_acc_account += (torch.argmax(eval_logits, dim=-1) == eval_target).sum().item()
                    eval_bce_loss = F.binary_cross_entropy(torch.sigmoid(eval_logits), F.one_hot(eval_target, num_classes=2).to(torch.float32))
                    ema_eval_loss = 0.9*ema_eval_loss + 0.1*eval_bce_loss
                    
                logging.warning(f"eval_ema_loss: {ema_eval_loss}, eval_acc: {total_acc_account/total_account}")
                model.train()

# step4 测试代码

In [5]:
model = GCNN()
# model = TextClassificationModel()
print("模型总参数：", sum(p.numel() for p in model.parameters()))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_iter = IMDB(root='.data', split='train')  # Dataset 类型的对象
train_data_loader = torch.utils.data.DataLoader(to_map_style_dataset(train_data_iter), batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

eval_data_iter = IMDB(root='.data', split='test')  # Dataset 类型的对象
eval_data_loader = torch.utils.data.DataLoader(to_map_style_dataset(eval_data_iter), batch_size=8, collate_fn=collate_fn)

train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, save_step_interval=500, eval_step_interval=300,
          save_path="./logs_imdb_text_classification", resume="")

模型总参数： 1214594
2022-11-23 23:51:51,286 (364936207:30) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.6079292297363281
2022-11-23 23:51:53,348 (364936207:30) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.6819902658462524
2022-11-23 23:51:55,257 (364936207:30) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.6360833048820496
2022-11-23 23:51:57,317 (364936207:30) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.5575909614562988
2022-11-23 23:51:59,271 (364936207:30) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.5020145773887634
2022-11-23 23:52:01,302 (364936207:30) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.551681637763977
2022-11-23 23:52:03,391 (364936207:30) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.4571375548839569
2022-11-23 23:52:05,506 (364936207:30) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.4281674325466156
2022-11-23 23:52:07,470 (364936207:30) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.465807080